## Install Necessary Libraries ##

In [ ]:
## ONCE DETECTRON2 IS INSTALLED, PLEASE COPY THE CONTENTS FROM THE SUBFOLDER DETECTRON2 FOLDER TO THE MAIN DETECTRON2 FOLDER!!!

# Upgrade pip
%pip install --upgrade pip

# Install OpenCV for image processing
%pip install opencv-python

# Install numpy at the desired version
%pip install numpy==1.26.1

# Install PyTorch and related packages (adjust versions/URLs as needed for your platform)
%pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.0+cu117 --index-url https://download.pytorch.org/whl/cu117

# Install gitpython and cython if not already installed
%pip install gitpython cython

# Install dependencies, but pin fvcore and pycocotools to compatible versions:
%pip install cython

# Install fvcore.
%pip install fvcore==0.1.5.post20221221

# Install pycocotools
%pip install pycocotools==2.0.8

# ---- Additional installations ----
%pip install docutils==0.19
%pip install sphinx==7
%pip install recommonmark==0.6.0
%pip install sphinx_rtd_theme
%pip install termcolor
%pip install yacs
%pip install tabulate
%pip install cloudpickle
%pip install future
# (The following two lines with CPU wheels are for Linux CP37; comment them out if not needed)
# %pip install https://download.pytorch.org/whl/cpu/torch-1.8.1%2Bcpu-cp37-cp37m-linux_x86_64.whl
# %pip install https://download.pytorch.org/whl/cpu/torchvision-0.9.1%2Bcpu-cp37-cp37m-linux_x86_64.whl
%pip install "omegaconf>=2.1.0.dev24"
%pip install "hydra-core>=1.1.0.dev5"
%pip install scipy
%pip install timm
# ---------------------------------

# Clone the Detectron2 repository and install it in editable mode
import os
if not os.path.exists('detectron2'):
    !git clone https://github.com/facebookresearch/detectron2.git

%cd detectron2
%pip install -e .
%cd ..

# Install TensorFlow and Keras for deep learning models
%pip install tensorflow

# Install pickleshare to fix the IPython warning
%pip install pickleshare

# Install additional required libraries
%pip install numpy matplotlib pandas scikit-learn tqdm Pillow

# Install geospatial libraries
%pip install geopandas rasterio shapely

# Install any other utility libraries if needed
%pip install jsonschema pyyaml

# Install IPython kernel to ensure compatibility
%pip install ipykernel

# After installations, you may need to restart the kernel
print("All libraries installed. Please restart the kernel to ensure all packages are loaded.")


## Import Necessary Libraries ##

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import copy
import random
import glob

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from yacs.config import CfgNode as CN
import detectron2.data.transforms as T
from detectron2.data import detection_utils as utils
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
print(np.__version__)


## Expanding Dataset (Do not run unless you need to make dataset bigger, the changes are permanent) ##

In [ ]:
# # Define the train folder and its subfolders
# train_folder = r"C:\Users\itagl\Downloads\AerialImageDataset\train"
# images_folder = os.path.join(train_folder, "images")
# gt_folder = os.path.join(train_folder, "gt")

# # Get a list of all TIF images in the images folder
# image_paths = glob.glob(os.path.join(images_folder, "*.tif"))
# print(f"Found {len(image_paths)} images in {images_folder}.")

# def rotate_image(img, angle):
#     """Rotate image by given angle (90, 180, 270 degrees)."""
#     if angle == 90:
#         return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
#     elif angle == 180:
#         return cv2.rotate(img, cv2.ROTATE_180)
#     elif angle == 270:
#         return cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
#     else:
#         raise ValueError("Unsupported rotation angle: " + str(angle))

# for img_path in image_paths:
#     base = os.path.splitext(os.path.basename(img_path))[0]  # e.g. 'austin1'
#     ext = os.path.splitext(img_path)[1]  # e.g. '.tif'
#     # Assume corresponding mask has the same base name in the gt folder
#     mask_path = os.path.join(gt_folder, base + ext)
    
#     # Read the original image and mask
#     img = cv2.imread(img_path)
#     mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)  # keep mask as-is
#     if img is None or mask is None:
#         print(f"Skipping {base}{ext}: cannot load image or mask.")
#         continue
    
#     # 1. Rotate 90
#     img_r90 = rotate_image(img, 90)
#     mask_r90 = rotate_image(mask, 90)
    
#     # 2. Rotate 180
#     img_r180 = rotate_image(img, 180)
#     mask_r180 = rotate_image(mask, 180)
    
#     # 3. Rotate 270
#     img_r270 = rotate_image(img, 270)
#     mask_r270 = rotate_image(mask, 270)
    
#     # 4. Mirror (flip horizontally)
#     img_m = cv2.flip(img, 1)
#     mask_m = cv2.flip(mask, 1)
    
#     # 5. Rotate 90 then mirror
#     img_r90m = cv2.flip(img_r90, 1)
#     mask_r90m = cv2.flip(mask_r90, 1)
    
#     # 6. Rotate 180 then mirror
#     img_r180m = cv2.flip(img_r180, 1)
#     mask_r180m = cv2.flip(mask_r180, 1)
    
#     # 7. Rotate 270 then mirror
#     img_r270m = cv2.flip(img_r270, 1)
#     mask_r270m = cv2.flip(mask_r270, 1)
    
#     # Save new images in the images folder
#     cv2.imwrite(os.path.join(images_folder, f"{base}r90{ext}"), img_r90)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r180{ext}"), img_r180)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r270{ext}"), img_r270)
#     cv2.imwrite(os.path.join(images_folder, f"{base}m{ext}"), img_m)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r90m{ext}"), img_r90m)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r180m{ext}"), img_r180m)
#     cv2.imwrite(os.path.join(images_folder, f"{base}r270m{ext}"), img_r270m)
    
#     # Save new masks in the gt folder
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r90{ext}"), mask_r90)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r180{ext}"), mask_r180)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r270{ext}"), mask_r270)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}m{ext}"), mask_m)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r90m{ext}"), mask_r90m)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r180m{ext}"), mask_r180m)
#     cv2.imwrite(os.path.join(gt_folder, f"{base}r270m{ext}"), mask_r270m)
    
#     print(f"Created 7 copies for {base}{ext}")

# print("Done creating rotated and mirrored copies.")

## Implement get_inria_dicts() ##

In [ ]:
def get_inria_dicts_split(img_dir, mask_dir, split="train", train_ratio=0.8):
    """
    Returns a list of dataset dictionaries for Detectron2.
    Splits the dataset into a training set (80% by default)
    and a validation set (20% by default) based on the 'split' parameter.
    
    Assumes each image in img_dir has a corresponding mask in mask_dir
    with the same filename.
    
    Parameters:
      - img_dir: Directory containing the images.
      - mask_dir: Directory containing the corresponding masks.
      - split: "train" or "val" to specify the subset.
      - train_ratio: Proportion of images for training.
    
    Returns:
      - List of dictionaries with image paths and annotations.
    """
    # Get list of image filenames (filter for common image extensions)
    image_files = [f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".png", ".jpeg", ".tif", ".tiff"))]
    
    # Shuffle the list for randomness (set seed if reproducibility is needed)
    random.shuffle(image_files)
    
    # Calculate the split index
    split_index = int(len(image_files) * train_ratio)
    
    # Select files based on split
    if split == "train":
        selected_files = image_files[:split_index]
    else:  # "val"
        selected_files = image_files[split_index:]
    
    dataset_dicts = []
    for idx, image_file in enumerate(selected_files):
        record = {}
        img_path = os.path.join(img_dir, image_file)
        base_name = os.path.splitext(image_file)[0]
        mask_path = os.path.join(mask_dir, image_file)  # same filename for mask
        
        # Read the image to get its dimensions
        img = cv2.imread(img_path)
        if img is None:
            continue
        height, width = img.shape[:2]
        
        record["file_name"] = img_path
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        
        # Read the corresponding mask in grayscale
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue
        
        # Threshold the mask to ensure it's binary
        _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        objs = []
        for cnt in contours:
            if cv2.contourArea(cnt) < 10:
                continue
            poly = cnt.flatten().tolist()
            if len(poly) < 6:
                continue
            x, y, w, h = cv2.boundingRect(cnt)
            objs.append({
                "bbox": [x, y, w, h],
                "bbox_mode": BoxMode.XYWH_ABS,
                "segmentation": [poly],
                "category_id": 0
            })
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts


## Register the dataset for Detectron2 ##

In [ ]:
# Register the dataset with Detectron2
# Define the paths to your image and mask folders
train_img_dir = r"C:\Users\itagl\Downloads\AerialImageDataset\train\images"
train_mask_dir = r"C:\Users\itagl\Downloads\AerialImageDataset\train\gt"

# Register training subset (80% of the image-mask pairs)
DatasetCatalog.register("inria_train", lambda: get_inria_dicts_split(train_img_dir, train_mask_dir, split="train"))
MetadataCatalog.get("inria_train").set(thing_classes=["rooftop"])

# Register validation subset (20% of the image-mask pairs)
DatasetCatalog.register("inria_val", lambda: get_inria_dicts_split(train_img_dir, train_mask_dir, split="val"))
MetadataCatalog.get("inria_val").set(thing_classes=["rooftop"])

print("Registered 'inria_train' and 'inria_val' datasets.")


## Visualize sample data to ensure it's loaded correctly ##

In [ ]:
# Visualize a few samples from the dataset
dataset_dicts = DatasetCatalog.get("inria_train")
num_samples = 3

for d in random.sample(dataset_dicts, num_samples):
    img = cv2.imread(d["file_name"])
    if img is None:
        continue
    visualizer = Visualizer(img[:, :, ::-1], MetadataCatalog.get("inria_train"), scale=1.0)
    out = visualizer.draw_dataset_dict(d)
    plt.figure(figsize=(10, 10))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.title("Sample Data Visualization")
    plt.axis("off")
    plt.show()


## Configure the Mask R-CNN model ##

In [ ]:
# Create configuration
cfg = get_cfg()
# Load a Mask R-CNN config from the model zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))

# Specify the training and validation datasets
cfg.DATASETS.TRAIN = ("inria_train",)
cfg.DATASETS.TEST = ("inria_val",)  # used for evaluation during training

# DataLoader and augmentation settings
cfg.DATALOADER.NUM_WORKERS = 12   # adjust based on your CPU capabilities

# Load pretrained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")

# Solver settings (adjust iterations and learning rate as needed)
cfg.SOLVER.IMS_PER_BATCH = 6    # images per batch
cfg.SOLVER.BASE_LR = 0.00025    # learning rate
cfg.SOLVER.MAX_ITER = 15000      # number of iterations, adjust based on convergence
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  # number of proposals per image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only one class: "rooftop"

# Optional: Enable built-in random cropping to 4000x4000 if desired
cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.CROP.TYPE = "absolute"
cfg.INPUT.CROP.SIZE = [2500, 2500]  # crop size (height, width)

# Set the output directory for model checkpoints and logs
cfg.OUTPUT_DIR = "./output_inria"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)




## Train the model ##

In [ ]:
# Initialize the trainer with the above configuration
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Validate Model on Validation set (You can skip this unless debugging) ##


In [ ]:
# # Validate the model on the validation set
# evaluator = COCOEvaluator("inria_val", cfg, False, output_dir=cfg.OUTPUT_DIR)
# val_loader = build_detection_test_loader(cfg, "inria_val")
# print(inference_on_dataset(trainer.model, val_loader, evaluator))

## Randomly select test image ##

In [ ]:
# Change to you path for test images
test_images_dir = r"C:\Users\maxxr\Downloads\Pics\Pics"

# Valid image extensions
img_extensions = [".jpg", ".jpeg", ".png", ".tif"]
all_test_files = [
    f for f in os.listdir(test_images_dir)
    if os.path.splitext(f)[1].lower() in img_extensions
]

if not all_test_files:
    raise ValueError("No valid images found in test_images_dir!")

# Randomly pick one
chosen_file = random.choice(all_test_files)
test_image_path = os.path.join(test_images_dir, chosen_file)

print("Randomly chosen test image:", test_image_path)


## Perform inference using the trained model and visualize ##

In [ ]:
# Update the configuration to use the final model weights
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4   # Set the confidence threshold
cfg.TEST.DETECTIONS_PER_IMAGE = 1000

predictor = DefaultPredictor(cfg)

# Load the test image
im = cv2.imread(test_image_path)

outputs = predictor(im)
print("Bounding boxes:", outputs["instances"].pred_boxes)
print("Segmentation masks:", outputs["instances"].pred_masks)

# Visualize the prediction
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get("inria_train"), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize=(12, 12))
plt.imshow(out.get_image()[:, :, ::-1])
plt.title("Inference Result")
plt.axis("off")
plt.show()


## Saving mask as an image ##


In [ ]:
outputs = predictor(im)
# Access the instance masks (one per detected instance)
instance_masks = outputs["instances"].pred_masks  # shape: (N, H, W)

# Move masks to CPU and convert to numpy
masks = instance_masks.to("cpu").numpy()  # shape: (N, H, W)

# Create an empty mask (initialized to zeros - black)
combined_mask = np.zeros((masks.shape[1], masks.shape[2]), dtype=np.uint8)

# Combine all masks: set pixel to 255 (white) if it belongs to any instance.
for mask in masks:
    # Ensure the mask is binary (e.g., >0.5 as threshold)
    combined_mask[mask > 0.5] = 255

# Optionally, save the combined mask image:
cv2.imwrite("predicted_mask.png", combined_mask)

## Perform obstacle detection using edge detection and color thresholding ##

In [ ]:
# Paths to the image and segmentation mask
image_path = test_image_path
mask_path = "predicted_mask.png"

# Physical dimensions of the picture in meters
physical_width_m = 986.58
physical_height_m = 708.28

if os.path.exists(image_path) and os.path.exists(mask_path):
    print("Files exist")
    # Load the original image and segmentation mask (houses in white, background in black)
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, 0)
    
    # Create an empty final mask (all black) to hold the processed usable roof areas
    final_mask = np.zeros_like(mask)
    
    # Find all house contours (each white region in the segmentation mask is assumed to be a house)
    house_contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if house_contours:
        for house_contour in house_contours:
            # Calculate bounding box for the current house
            x, y, w, h = cv2.boundingRect(house_contour)
            
            # Zoom into the roof and corresponding mask area using the original mask
            largest_roof = image[y:y+h, x:x+w]
            roof_mask = mask[y:y+h, x:x+w]
            
            # Apply the mask to the roof area to isolate it
            masked_roof = cv2.bitwise_and(largest_roof, largest_roof, mask=roof_mask)
            
            # Convert masked roof to grayscale and then to binary
            gray_masked_roof = cv2.cvtColor(masked_roof, cv2.COLOR_BGR2GRAY)
            blurred_roof = cv2.GaussianBlur(gray_masked_roof, (5, 5), 0)
            _, binary_mask = cv2.threshold(blurred_roof, 50, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            # Find contours within the masked area from the binary mask
            contours_in_roof, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Create an image to draw the filled contours (obstacles)
            filled_contours_img = np.zeros_like(masked_roof)
            
            # Define area thresholds to filter out false detections:
            min_obstacle_area = 50
            max_obstacle_area = (w * h) * 0.3  # Adjust this fraction based on your data
            
            filtered_contours = []
            for cnt in contours_in_roof:
                area = cv2.contourArea(cnt)
                if min_obstacle_area < area < max_obstacle_area:
                    filtered_contours.append(cnt)
            
            # Draw and fill the filtered contours with green (using your exact code)
            cv2.drawContours(filled_contours_img, filtered_contours, -1, (0, 255, 0), thickness=cv2.FILLED)
            
            # Convert the filled contours image to grayscale, then threshold to obtain a binary obstacle mask
            filled_contours_gray = cv2.cvtColor(filled_contours_img, cv2.COLOR_BGR2GRAY)
            _, obstacle_mask = cv2.threshold(filled_contours_gray, 1, 255, cv2.THRESH_BINARY)
            # Invert the obstacle mask so that obstacles become black
            obstacle_mask_inv = cv2.bitwise_not(obstacle_mask)
            
            # Remove obstacles from the original roof mask (usable roof area remains white)
            processed_roof_mask = cv2.bitwise_and(roof_mask, obstacle_mask_inv)
            
            # Merge the processed roof mask into the final mask at the proper location
            final_mask[y:y+h, x:x+w] = cv2.bitwise_or(final_mask[y:y+h, x:x+w], processed_roof_mask)
        
        # Count white pixels in both the original and processed masks
        original_white_pixels = cv2.countNonZero(mask)
        processed_white_pixels = cv2.countNonZero(final_mask)
        print("White pixels in original segmentation mask:", original_white_pixels)
        print("White pixels in processed mask (obstacles removed):", processed_white_pixels)
        
        # Get the pixel dimensions of the mask
        mask_height, mask_width = final_mask.shape  # height and width in pixels
        
        # Calculate the area each pixel represents.
        # Each pixel's physical width: physical_width_m / mask_width
        # Each pixel's physical height: physical_height_m / mask_height
        area_per_pixel = (physical_width_m / mask_width) * (physical_height_m / mask_height)
        
        # Total white area (usable roof area) in square meters and square kilometers:
        white_area_sqm = processed_white_pixels * area_per_pixel
        white_area_sqkm = white_area_sqm / 1e6
        
        print("White area in square meters: {:.2f} m²".format(white_area_sqm))
        print("White area in square kilometers: {:.3f} km²".format(white_area_sqkm))
        
        # Display the original segmentation mask and the final processed mask side by side
        plt.figure(figsize=(14, 7))
        plt.subplot(1, 2, 1)
        plt.imshow(mask, cmap='gray')
        plt.title("Original Segmentation Mask")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(final_mask, cmap='gray')
        plt.title("Processed Mask (Obstacles Removed)")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    else:
        print("No houses detected in the segmentation mask.")
else:
    print("Files do not exist, check the paths")


## Calculate the effective roof area excluding obstacles ##

In [ ]:
# Assume roof_mask is a binary mask of the roof area
roof_mask = cv2.imread('/Users/ericdayan/Documents/Documents/University/Capstone/Object Detection.coco/train/img2736_png.rf.58fc1effe9f0eaa92764b325baf93979.jpg', cv2.IMREAD_GRAYSCALE)
roof_area_pixels = cv2.countNonZero(roof_mask)

# Obstacle area in pixels
obstacle_area_pixels = cv2.countNonZero(combined_mask)

# Effective area in pixels
effective_area_pixels = roof_area_pixels - obstacle_area_pixels

# Spatial resolution in meters per pixel (you need to define this based on your data)
resolution = 0.2  # Example: each pixel represents 0.1 meters

# Convert to square meters
effective_area_m2 = effective_area_pixels * (resolution ** 2)

print(f"Effective roof area available for solar panels: {effective_area_m2:.2f} square meters")